In [111]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
import seaborn as sns 
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer,make_column_selector
from sklearn.impute import SimpleImputer 

In [112]:
os.chdir(r"C:\Users\Administrator.DAI-PC2\Desktop\jupiter_demo\ML\Day15\analticsVidya_bigMart")

train = pd.read_csv("train_v9rqX0R.csv")
X_train = train.drop(['Outlet_Identifier', 'Item_Outlet_Sales'], axis=1)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Size                6113 non-null   object 
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 666.0+ KB


In [113]:
############### Cleaning 
X_train.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [114]:
X_train['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

In [18]:
X_train['Item_Fat_Content'].replace(to_replace = ['LF','low fat','reg'], value=['Low Fat', 'Low Fat','Regular'], inplace=True)

In [19]:
X_train['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [23]:
test = pd.read_csv("test_AbJTz2l.csv")
test.drop(['Outlet_Identifier'], axis=1, inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                4705 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Establishment_Year  5681 non-null   int64  
 7   Outlet_Size                4075 non-null   object 
 8   Outlet_Location_Type       5681 non-null   object 
 9   Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 444.0+ KB


In [25]:
test['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    3396
Regular    1935
LF          206
reg          78
low fat      66
Name: count, dtype: int64

In [27]:
test['Item_Fat_Content'].replace(to_replace = ['LF','low fat','reg'], value=['Low Fat', 'Low Fat','Regular'], inplace=True)

In [28]:
test['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    3668
Regular    2013
Name: count, dtype: int64

Data cleaning on the item_identifier and item_weight -- Train set

In [42]:
X_weight_iden = X_train.iloc[:,0:2]

In [83]:
item = X_weight_iden.drop_duplicates() #drop duplicates  #dropna

In [84]:
item = item.dropna(axis=0)

In [87]:
item

,Item_Identifier,Item_Weight
0,FDA15,9.300
1,DRC01,5.920
2,FDN15,17.500
3,FDX07,19.200
4,NCD19,8.930
...,...,...
7298,NCW05,20.250
7373,FDS09,8.895
7421,FDU43,19.350
7944,FDO49,10.600


In [88]:
X_train_new = pd.merge(item, X_train,on='Item_Identifier')

In [99]:
X_train_new.drop('Item_Weight_y',axis=1,inplace=True)

Data cleaning on the item_identifier and item_weight -- Test set

In [100]:
X_weight_iden_y = test.iloc[:,0:2]

In [101]:
item_y = X_weight_iden.drop_duplicates() #drop duplicates  #dropna

In [102]:
item_y = item.dropna(axis=0)

In [103]:
item_y

,Item_Identifier,Item_Weight
0,FDA15,9.300
1,DRC01,5.920
2,FDN15,17.500
3,FDX07,19.200
4,NCD19,8.930
...,...,...
7298,NCW05,20.250
7373,FDS09,8.895
7421,FDU43,19.350
7944,FDO49,10.600


In [104]:
X_test_new = pd.merge(item, test,on='Item_Identifier')

In [108]:
X_test_new.isnull().sum()

Item_Identifier                 0
Item_Weight_x                   0
Item_Weight_y                 975
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Establishment_Year       0
Outlet_Size                  1596
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [109]:
X_test_new.drop('Item_Weight_y',axis=1,inplace=True)

In [110]:
X_test_new

Item_Identifier                 0
Item_Weight_x                   0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Establishment_Year       0
Outlet_Size                  1596
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [115]:
X_weight_iden

,Item_Identifier,Item_Weight
0,FDA15,9.30
1,DRC01,5.92
2,FDN15,17.50
3,FDX07,19.20
4,NCD19,8.93
...,...,...
8471,NCT53,5.40
8472,NCS41,NaN
8480,FDQ58,NaN
8487,DRG13,NaN


In [116]:
X_weight_iden_y

,Item_Identifier,Item_Weight
0,FDW58,20.750
1,FDW14,8.300
2,NCN55,14.600
3,FDQ58,7.315
4,FDY38,NaN
...,...,...
5676,FDB58,10.500
5677,FDD47,7.600
5678,NCO17,10.000
5679,FDJ26,15.300


In [118]:
item = pd.merge(X_weight_iden,X_weight_iden_y, how='outer')

In [120]:
item.isna().sum()

Item_Identifier       0
Item_Weight        1690
dtype: int64

In [121]:
item = item.drop_duplicates()

In [122]:
item = item.dropna(axis=0)

In [124]:
item

,Item_Identifier,Item_Weight
0,FDA15,9.300
1,DRC01,5.920
4,FDN15,17.500
7,FDX07,19.200
10,NCD19,8.930
...,...,...
5838,NCT53,5.400
5864,FDE52,10.395
5925,FDN52,9.395
5940,FDQ60,6.195


In [126]:
X_train.drop('Item_Weight',inplace=True,axis=1)

In [127]:
X_train_ip = pd.merge(item,X_train, how='inner', on='Item_Identifier')

In [129]:
X_train_ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Size                6113 non-null   object 
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 666.0+ KB


In [130]:
test.drop('Item_Weight',inplace=True,axis=1)

In [131]:
X_test_ip = pd.merge(item,test, how='inner', on='Item_Identifier')

In [132]:
X_test_ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                5681 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Establishment_Year  5681 non-null   int64  
 7   Outlet_Size                4075 non-null   object 
 8   Outlet_Location_Type       5681 non-null   object 
 9   Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 444.0+ KB


In [145]:
X_train_outlet = train.drop('Item_Outlet_Sales', axis=1).loc[:,'Outlet_Identifier':'Outlet_Type']

In [146]:
X_train_outlet

,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,OUT010,1998,NaN,Tier 3,Grocery Store
4,OUT013,1987,High,Tier 3,Supermarket Type1
...,...,...,...,...,...
8518,OUT013,1987,High,Tier 3,Supermarket Type1
8519,OUT045,2002,NaN,Tier 2,Supermarket Type1
8520,OUT035,2004,Small,Tier 2,Supermarket Type1
8521,OUT018,2009,Medium,Tier 3,Supermarket Type2


In [136]:
X_train_outlet = test.drop('Item_Outlet_Sales', axis=1).loc[:,'Outlet_Identifier':'Outlet_Type']

,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1999,Medium,Tier 1,Supermarket Type1
1,2007,NaN,Tier 2,Supermarket Type1
2,1998,NaN,Tier 3,Grocery Store
3,2007,NaN,Tier 2,Supermarket Type1
4,1985,Medium,Tier 3,Supermarket Type3
...,...,...,...,...
5676,1997,Small,Tier 1,Supermarket Type1
5677,2009,Medium,Tier 3,Supermarket Type2
5678,2002,NaN,Tier 2,Supermarket Type1
5679,2007,NaN,Tier 2,Supermarket Type1


In [148]:
test = pd.read_csv("test_AbJTz2l.csv")

In [149]:
X_test_outlet = test.loc[:,'Outlet_Identifier':'Outlet_Type']

In [150]:
X_train_test_outlet = pd.merge(X_train_outlet, X_test_outlet, how='outer')

In [155]:
X_train_test_outlet = X_train_test_outlet.drop_duplicates()

In [156]:
X_train_test_outlet['Outlet_Size'].fillna(value='small', inplace=True)

,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,OUT049,1999,Medium,Tier 1,Supermarket Type1
576600,OUT018,2009,Medium,Tier 3,Supermarket Type2
1150104,OUT010,1998,NaN,Tier 3,Grocery Store
1355454,OUT013,1987,High,Tier 3,Supermarket Type1
1934226,OUT027,1985,Medium,Tier 3,Supermarket Type3
2517666,OUT045,2002,NaN,Tier 2,Supermarket Type1
3092717,OUT017,2007,NaN,Tier 2,Supermarket Type1
3664059,OUT046,1997,Small,Tier 1,Supermarket Type1
4240659,OUT035,2004,Small,Tier 2,Supermarket Type1
4817259,OUT019,1985,Small,Tier 1,Grocery Store


In [220]:
X_train_test_outlet_bayes['Outlet_Size'].fillna(value='Small')

1355454      High
0          Medium
576600     Medium
1934226    Medium
3664059     Small
4240659     Small
4817259     Small
1150104     Small
2517666     Small
3092717     Small
Name: Outlet_Size, dtype: object

In [221]:
X_test_final=X_test_ip.fillna(value='Small')

In [223]:
X_test_final

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,9.300,Low Fat,0.016022,Dairy,249.9092,1997,Small,Tier 1,Supermarket Type1
1,DRC01,5.920,Regular,0.032137,Soft Drinks,49.1692,1998,Small,Tier 3,Grocery Store
2,DRC01,5.920,Regular,0.019196,Soft Drinks,50.3692,2004,Small,Tier 2,Supermarket Type1
3,DRC01,5.920,Regular,0.019230,Soft Drinks,47.5692,1999,Medium,Tier 1,Supermarket Type1
4,FDN15,17.500,Low Fat,0.016731,Meat,139.0180,2004,Small,Tier 2,Supermarket Type1
...,...,...,...,...,...,...,...,...,...,...
5676,FDK57,10.195,Low Fat,0.080456,Snack Foods,120.1440,2002,Small,Tier 2,Supermarket Type1
5677,FDK57,10.195,Low Fat,0.080747,Snack Foods,120.5440,2007,Small,Tier 2,Supermarket Type1
5678,FDK57,10.195,Low Fat,0.080226,Snack Foods,121.2440,1987,High,Tier 3,Supermarket Type1
5679,FDK57,10.195,Low Fat,0.000000,Snack Foods,119.7440,1998,Small,Tier 3,Grocery Store


In [222]:
X_train_final=X_test_ip.fillna(value='Small')

In [224]:
X_train_final

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,9.300,Low Fat,0.016022,Dairy,249.9092,1997,Small,Tier 1,Supermarket Type1
1,DRC01,5.920,Regular,0.032137,Soft Drinks,49.1692,1998,Small,Tier 3,Grocery Store
2,DRC01,5.920,Regular,0.019196,Soft Drinks,50.3692,2004,Small,Tier 2,Supermarket Type1
3,DRC01,5.920,Regular,0.019230,Soft Drinks,47.5692,1999,Medium,Tier 1,Supermarket Type1
4,FDN15,17.500,Low Fat,0.016731,Meat,139.0180,2004,Small,Tier 2,Supermarket Type1
...,...,...,...,...,...,...,...,...,...,...
5676,FDK57,10.195,Low Fat,0.080456,Snack Foods,120.1440,2002,Small,Tier 2,Supermarket Type1
5677,FDK57,10.195,Low Fat,0.080747,Snack Foods,120.5440,2007,Small,Tier 2,Supermarket Type1
5678,FDK57,10.195,Low Fat,0.080226,Snack Foods,121.2440,1987,High,Tier 3,Supermarket Type1
5679,FDK57,10.195,Low Fat,0.000000,Snack Foods,119.7440,1998,Small,Tier 3,Grocery Store


In [235]:
from xgboost import XGBRegressor as XGBoost

In [236]:
df_trn = pd.get_dummies(X_train_final, drop_first=True)

In [237]:
df_tst = pd.get_dummies(X_test_final, drop_first=True)

In [238]:
from sklearn.model_selection import KFold, GridSearchCV

In [239]:
kfold = KFold(n_splits=5, shuffle=True,
                        random_state=24)

In [241]:
train_x = pd.read_csv("train_v9rqX0R.csv")

In [242]:
train_y = train_x['Item_Outlet_Sales']

In [243]:
params = {'learning_rate':np.linspace(0.001,0.9,10),
          'max_depth':[None,2,3,4],
          'n_estimators':[25,50,100]}

In [245]:

gcv = GridSearchCV(XGBoost, param_grid=params, cv=kfold ,
                   scoring ='neg_root_mean_squared_error')
gcv.fit(df_trn, train_y)
print(gcv.best_params_)
print(gcv.best_score_)

ValueError: Found input variables with inconsistent numbers of samples: [5681, 8523]

In [158]:
X_train_test_outlet_bayes = X_train_test_outlet.iloc[:,2:]

In [157]:
from sklearn.naive_bayes import BernoulliNB

In [162]:
X_train_test_outlet_bayes.sort_values(by='Outlet_Size',inplace=True)

In [163]:
bayes_train = X_train_test_outlet_bayes.iloc[:-3,:]

In [169]:
bayes_train

,Outlet_Size,Outlet_Location_Type,Outlet_Type
1355454,High,Tier 3,Supermarket Type1
0,Medium,Tier 1,Supermarket Type1
576600,Medium,Tier 3,Supermarket Type2
1934226,Medium,Tier 3,Supermarket Type3
3664059,Small,Tier 1,Supermarket Type1
4240659,Small,Tier 2,Supermarket Type1
4817259,Small,Tier 1,Grocery Store


In [190]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [191]:
ohc = OneHotEncoder()

In [192]:
le = LabelEncoder()

In [198]:
le_fit = le.fit_transform(bayes_train.iloc[:,0])

In [201]:
le_fit

array([0, 1, 1, 1, 2, 2, 2])

In [199]:
y_transformed = ohc.fit_transform(bayes_train.iloc[:,1:])

In [204]:
y_transformed.shape

(7, 7)

In [212]:
BernoulliNB().fit(y_transformed,le_fit)

BernoulliNB()

In [213]:
X_train_test_outlet_bayes.iloc[-3:,1:]

,Outlet_Location_Type,Outlet_Type
1150104,Tier 3,Grocery Store
2517666,Tier 2,Supermarket Type1
3092717,Tier 2,Supermarket Type1


In [214]:
y_transformed = ohc.fit_transform(bayes_test.iloc[:,1:])

In [215]:
bayes_test = X_train_test_outlet_bayes.iloc[-3:,1:]

In [216]:
bayes_test

,Outlet_Location_Type,Outlet_Type
1150104,Tier 3,Grocery Store
2517666,Tier 2,Supermarket Type1
3092717,Tier 2,Supermarket Type1


In [217]:
y_transformed_tst = ohc.fit_transform(bayes_test.iloc[:,1:])

In [218]:
BernoulliNB().predict(y_transformed_tst)

NotFittedError: This BernoulliNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [219]:
bayes_test_dum = pd.get_dummies(bayes_test, drop_first=True)

In [173]:
bayes_test_dum

,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1
1150104,True,False
2517666,False,True
3092717,False,True


In [180]:
nb = BernoulliNB()

In [184]:
nb.fit(bayes_train_dum.iloc[:,-2:],bayes_train[])

ValueError: y should be a 1d array, got an array of shape (7, 7) instead.

In [183]:
nb.predict(bayes_test_dum)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Outlet_Location_Type_Tier 3
- Outlet_Type_Supermarket Type1
Feature names seen at fit time, yet now missing:
- Outlet_Type_Supermarket Type2
- Outlet_Type_Supermarket Type3
